In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp
import os
import time

# 1. Landmarking using mediapipe holistic

In [2]:
holistics = mp.solutions.holistic # To bring our holistic model
drawing = mp.solutions.drawing_utils # Use fot drawing the utilities

In [3]:
def Detection(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    img.flags.writeable = False                  # Image is no longer writeable
    results = model.process(img)                 # Make prediction
    img.flags.writeable = True                   # Image is now writeable 
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return img, results

In [4]:
def DrawingCustomLandmarks(image, results):
    # Draw face connections
    drawing.draw_landmarks(image, results.face_landmarks, holistics.FACEMESH_TESSELATION, 
                             drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    drawing.draw_landmarks(image, results.pose_landmarks, holistics.POSE_CONNECTIONS,
                             drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    drawing.draw_landmarks(image, results.left_hand_landmarks, holistics.HAND_CONNECTIONS, 
                             drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    drawing.draw_landmarks(image, results.right_hand_landmarks, holistics.HAND_CONNECTIONS, 
                             drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 2. Extracting the Values

In [6]:
def ExtractingVals(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    left = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left, right])

In [8]:
np.load('New.npy')

array([ 0.59710538,  0.54454541, -1.31793547, ...,  0.        ,
        0.        ,  0.        ])

# 3. Data Collection

In [18]:
# Path for exported data, numpy arrays
DataPath = os.path.join('ExData') 

# Actions that we try to detect
actions = np.array(['Push Ups', 'Lunges', 'Squats','Sit Ups', 'High Knees'])

# Thirty videos worth of data
numOfSequences = 30

# Videos are going to be 30 frames in length
sequenceLength = 30

# Folder start
startFolder = 30

In [19]:
for action in actions: 
    for sequence in range(numOfSequences):
        try: 
            os.makedirs(os.path.join(DataPath, action, str(sequence)))
        except:
            pass

# 4. Training and Testing

In [22]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with holistics.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(numOfSequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequenceLength):
                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = Detection(frame, holistic)

                # Draw landmarks
                DrawingCustomLandmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'Starting Collection', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Window', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Window', image)
                
                # NEW Export keypoints
                keypoints = ExtractingVals(results)
                npy_path = os.path.join(DataPath, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [23]:
cap.release()
cv2.destroyAllWindows()

# 5. Preprocess Data

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'Push Ups': 0, 'Lunges': 1, 'Squats': 2, 'Sit Ups': 3, 'High Knees': 4}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in range(numOfSequences):
        window = []
        for frame_num in range(sequenceLength):
            res = np.load(os.path.join(DataPath, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
np.array(sequences).shape

(150, 30, 1662)

In [16]:
X = np.array(sequences)

In [17]:
X.shape

(150, 30, 1662)

In [18]:
y = to_categorical(labels).astype(int)

In [18]:
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0,

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
y_test.shape

(8, 5)

# 6.  Creating & Training LSTM Neural Network Architecture

In [8]:
# extra

from keras.models import load_model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.models import load_model

In [33]:
log_dir = os.path.join('LogsDirectory')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500


KeyboardInterrupt: 

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596,741
Trainable params: 596,741
Non-trai

In [ ]:
res = model.predict(X_test)

# 7. Make Predictions

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 8. Save Weights

In [ ]:
model.save('Excercise.h5')

In [9]:
model=load_model('Excercise.h5')

# 9. Evaluation using Confusion Matrix and Accuracy

In [35]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

5/5 [==============================] - 1s 50ms/step


In [36]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[114,   0],
        [  0,  28]],

       [[112,   2],
        [  2,  26]],

       [[110,   3],
        [  2,  27]],

       [[114,   1],
        [  1,  26]],

       [[112,   0],
        [  1,  29]]], dtype=int64)

In [ ]:
accuracy_score(ytrue, yhat)

# 10. Test in Real Time

In [10]:
colors = [(255,255,31), (117,245,16), (255,128,0), (28,255,248), (225,28,28), (0,204,204), (204,0,204), (16,117,245),  (0,0,204), (255,255,51)]
def probabilityVisualize(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [23]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with holistics.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = Detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        DrawingCustomLandmarks(image, results)
        
        # 2. Prediction logic
        keypoints = ExtractingVals(results)
        sequence.insert(0,keypoints)
        print(sequence)
        sequence = sequence[:30]        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Visualization logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

#             # Visualise probabilities
            image = probabilityVisualize(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Window', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.3846896 ,  0.59390414, -1.23835278, ...,  0.        ,
        0.        ,  0.        ])]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.36392766,  0.57244909, -1.35465765, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.3846896 ,  0.59390414, -1.23835278, ...,  0.        ,
        0.        ,  0.        ])]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.3247917 ,  0.57048255, -1.32835019, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36392766,  0.57244909, -1.35465765, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.3846896 ,  0.59390414, -1.23835278, ...,  0.        ,
        0.        ,  0.        ])]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.31765664,  0.56582808, -1.16425312, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.3247917 ,  0.57048255, -1.32835019, ...,  0.        ,
        0. 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.34804636,  0.56374747, -1.45774436, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.33791557,  0.56398654, -1.2169044 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.33692753,  0.56422448, -1.1185416 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.3484858 ,  0.56088966, -1.16397786, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.37775269,  0.56035256, -1.3027494 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.39757788,  0.55907327, -1.48665166, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.40239462,  0.55788332, -1.60910261, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.39640233,  0.55838984, -1.49205613, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.36947772,  0.56659949, -1.28074074, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.31765664,  0.56582808, -1.16425312, ..., 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.22780612,  0.57881415, -0.85595477, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22438546,  0.57965076, -0.6973123 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22808015,  0.56874382, -0.75503719, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.24708161,  0.56647152, -0.90532696, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34825563,  0.56619167, -1.30849624, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.35273817,  0.56591165, -1.43102086, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34804636,  0.56374747, -1.45774436, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.33791557,  0.56398654, -1.2169044 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.33692753,  0.56422448, -1.1185416 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.3484858 ,  0.56088966, -1.16397786, ..., 

[array([ 0.31972164,  0.54851782, -1.30197978, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.30452502,  0.55228496, -1.39564967, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.26208562,  0.57755369, -1.11913288, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.2295026 ,  0.57962984, -0.7710045 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22780612,  0.57881415, -0.85595477, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22438546,  0.57965076, -0.6973123 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22808015,  0.56874382, -0.75503719, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.24708161,  0.56647152, -0.90532696, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34825563,  0.56619167, -1.30849624, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.35273817,  0.56591165, -1.43102086, ...,  0.        ,
        0.        ,  0.        ]), array([ 0

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.31043559,  0.5547722 , -1.18219471, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.32177132,  0.54814327, -1.24003386, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.32360038,  0.54227555, -1.26003897, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.32627052,  0.5445171 , -1.31808639, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.31972164,  0.54851782, -1.30197978, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.30452502,  0.55228496, -1.39564967, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.26208562,  0.57755369, -1.11913288, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.2295026 ,  0.57962984, -0.7710045 , ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22780612,  0.57881415, -0.85595477, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22438546,  0.57965076, -0.6973123 , ..., 

...

1/1 [==============================] - 0s 53ms/step
High Knees
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.41638103,  0.48918101, -1.58613503, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41060743,  0.48928738, -1.58018279, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41074809,  0.48981121, -1.64109671, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.39492494,  0.497776  , -1.55541646, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34710476,  0.50511384, -1.45584083, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27915698,  0.50786179, -1.11414623, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27328426,  0.50793463, -0.91928273, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27263594,  0.50887746, -1.15468621, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27174279,  0.5114994 , -1.14599872, ...,  0.        ,
        0.        ,  0.

1/1 [==============================] - 0s 44ms/step
High Knees
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.41457215,  0.49159852, -1.59036708, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41450372,  0.49107936, -1.60274005, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.4127256 ,  0.48921517, -1.66078782, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41638103,  0.48918101, -1.58613503, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41060743,  0.48928738, -1.58018279, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41074809,  0.48981121, -1.64109671, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.39492494,  0.497776  , -1.55541646, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34710476,  0.50511384, -1.45584083, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27915698,  0.50786179, -1.11414623, ...,  0.        ,
        0.        ,  0.

1/1 [==============================] - 0s 55ms/step
High Knees
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.27141318,  0.54659712, -1.10876656, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34975657,  0.50516856, -1.22719276, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41563845,  0.49121207, -1.57160199, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41457215,  0.49159852, -1.59036708, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41450372,  0.49107936, -1.60274005, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.4127256 ,  0.48921517, -1.66078782, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41638103,  0.48918101, -1.58613503, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41060743,  0.48928738, -1.58018279, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41074809,  0.48981121, -1.64109671, ...,  0.        ,
        0.        ,  0.

1/1 [==============================] - 0s 67ms/step
High Knees
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.26548314,  0.55332845, -1.20867467, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.25146523,  0.55783802, -1.16342604, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.25571626,  0.56371647, -0.98549622, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27141318,  0.54659712, -1.10876656, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34975657,  0.50516856, -1.22719276, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41563845,  0.49121207, -1.57160199, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41457215,  0.49159852, -1.59036708, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41450372,  0.49107936, -1.60274005, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.4127256 ,  0.48921517, -1.66078782, ...,  0.        ,
        0.        ,  0.

1/1 [==============================] - 0s 67ms/step
High Knees
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[array([ 0.5199753 ,  0.55547488, -1.89990366, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.47728544,  0.54700941, -1.87034357, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.37042412,  0.54452968, -1.71612036, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.26548314,  0.55332845, -1.20867467, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.25146523,  0.55783802, -1.16342604, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.25571626,  0.56371647, -0.98549622, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.27141318,  0.54659712, -1.10876656, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.34975657,  0.50516856, -1.22719276, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.41563845,  0.49121207, -1.57160199, ...,  0.        ,
        0.        ,  0.

In [14]:
cap.release()
cv2.destroyAllWindows()